# Trabajo Final - Macroeconomía con Python

## Alumno: Alexander Max Tello Fuentes
## Código: 20156122

## Pregunta 1

In [14]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
pd.reset_option('^display.', silent=True)
df = pd.read_csv('wages.csv')
print (df)

      wage  educ  exper  tenure  nonwhite  female  married  northcen  south  \
0     3.10    11      2       0         0       1        0         0      0   
1     3.24    12     22       2         0       1        1         0      0   
2     3.00    11      2       0         0       0        0         0      0   
3     6.00     8     44      28         0       0        1         0      0   
4     5.30    12      7       2         0       0        1         0      0   
..     ...   ...    ...     ...       ...     ...      ...       ...    ...   
395   3.76     6      6       0         0       0        0         0      1   
396   3.10     6     14       0         1       1        0         0      1   
397   4.29    12     47      25         0       1        1         0      1   
398  10.92    12     34       5         0       0        1         0      1   
399   7.50    16      6       2         0       1        1         0      1   

     west  east  
0       1     0  
1       1     0

Trataremos de explicar los salarios a través de la participación de la educación, la experiencia, la tenencia, el sexo, el estado civil, etc. Para ello realizaremos la regresión de los salarios versus las variables explicativas mencionadas, utilizando Mínimos Cuadrados Restringidos (MCR).

a) Desde Cero - Sin restricciones

In [24]:
Y=df['wage']
x=pd.concat([df['educ'],df['exper'],df['tenure'],df['nonwhite'],df['female'],df['married'],df['northcen'],df['south'],df['west'],df['east']],axis=1)
X=np.column_stack((np.ones(400), x))
print (Y)
print (X)

0       3.10
1       3.24
2       3.00
3       6.00
4       5.30
       ...  
395     3.76
396     3.10
397     4.29
398    10.92
399     7.50
Name: wage, Length: 400, dtype: float64
[[ 1. 11.  2. ...  0.  1.  0.]
 [ 1. 12. 22. ...  0.  1.  0.]
 [ 1. 11.  2. ...  0.  1.  0.]
 ...
 [ 1. 12. 47. ...  1.  0.  0.]
 [ 1. 12. 34. ...  1.  0.  0.]
 [ 1. 16.  6. ...  1.  0.  0.]]


Usaremos linalg para calcular los beta. Así, resolvemos la operación p*b=z, donde debemos hallar b que es el vector beta.


In [26]:
np.set_printoptions(precision=5, suppress=True)
p= np.dot(X.T, X)
z= np.dot(X.T, Y)
b= np.linalg.solve(p,z)
print(b)
print('Verify: "p b - z = 0"')
print(np.dot(p,b) - z.T)


[ 10.19095   0.57925   0.02942   0.15504   0.01121  -1.82068   0.36203
 -12.4235  -12.51717 -10.94049 -11.90172]
Verify: "p b - z = 0"
[ 0.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0.]


Ahora calculamos los residuos para hallar los errores estándar del b ols.

1) Hallamos los residuos U

In [5]:
U=Y-np.dot(X,b)
print(U)

0     -0.760366
1     -2.460170
2     -2.681043
3     -3.882226
4     -1.779518
         ...   
395    2.434200
396    3.348290
397   -4.135001
398    4.157649
399    1.530268
Name: wage, Length: 400, dtype: float64


In [6]:
n=400
k=11
V=np.dot(np.linalg.inv(p),np.dot(U.T,U))/(n-k-1)
print(V)

[[ 5.97681e+14 -9.98205e-02 -1.81010e-03 -2.72796e-02 -1.69203e-01
  -1.85045e-01 -1.08091e-02 -5.97681e+14 -5.97681e+14 -5.97681e+14
  -5.97681e+14]
 [-8.56048e-02  3.67950e-03  2.93803e-04 -9.08150e-05  2.73242e-03
   1.45185e-03 -3.34014e-03  3.46282e-02  4.55534e-02  4.72920e-02
   4.39857e-02]
 [-1.50009e-02  2.93803e-04  2.03664e-04 -1.68349e-04  3.98861e-05
  -6.32697e-05 -1.30226e-03  9.31465e-03  1.04209e-02  1.10022e-02
   1.06567e-02]
 [ 7.46459e-03 -9.08150e-05 -1.68349e-04  6.27343e-04  2.47167e-04
   1.05499e-03 -7.58700e-04 -6.35614e-03 -6.04661e-03 -7.51121e-03
  -7.40411e-03]
 [-4.67485e-02  2.73242e-03  3.98861e-05  2.47167e-04  2.58090e-01
   2.49002e-03  3.99255e-03 -4.38516e-03 -2.62477e-02 -1.63847e-02
  -1.83789e-02]
 [-1.11976e-01  1.45185e-03 -6.32697e-05  1.05499e-03  2.49002e-03
   9.44114e-02  9.69526e-03  3.86265e-02  3.79083e-02  2.40373e-02
   3.98020e-02]
 [ 1.83430e-02 -3.34014e-03 -1.30226e-03 -7.58700e-04  3.99255e-03
   9.69526e-03  1.10427e-01 -3.11

De la matriz V, extraemos la diagonal principal y le aplicamos raíz cuadrada a cada elemento.

In [7]:
e=np.diagonal(V)**(1/2)
print(e)

[24447507.13095        0.06066        0.01427        0.02505
        0.50803        0.30726        0.33231 24447507.13095
 24447507.13095 24447507.13095 24447507.13095]


b) Desde cero: con restricciones

In [8]:
R =np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1])[np.newaxis]
print(R)
print(R.T)

[[0 0 0 0 0 0 0 1 1 1 1]]
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]


Ahora calculamos los "brls"

In [9]:
init=np.dot(np.linalg.inv(p),R.T)
m=np.dot(np.dot(R,np.linalg.inv(p)),R.T)
minv=np.linalg.inv(m)
r=np.dot(R,b)
print(init)
print (minv)
print (r)

[[-2.67780e+14]
 [ 1.92048e-02]
 [ 4.63652e-03]
 [-3.05985e-03]
 [-7.32493e-03]
 [ 1.57230e-02]
 [-1.18627e-02]
 [ 2.67780e+14]
 [ 2.67780e+14]
 [ 2.67780e+14]
 [ 2.67780e+14]]
[[0.]]
[-47.78287]


brls es el estimador de los mínimos cuadrados restringidos que se estima a continuación:

In [10]:
brls=b+np.dot(np.dot(init, minv),-r) 
print(brls)

[-1.75477  0.57925  0.02942  0.15504  0.01121 -1.82068  0.36203 -0.47778
 -0.57145  1.00523  0.044  ]


Ahora calculamos los residuos para obtener los errores estándar de brls

In [11]:
urls=Y-np.dot(X,brls)
print(urls)

0     -0.760366
1     -2.460170
2     -2.681043
3     -3.882226
4     -1.779518
         ...   
395    2.434200
396    3.348290
397   -4.135001
398    4.157649
399    1.530268
Name: wage, Length: 400, dtype: float64


Calculamos la matriz de varianzas y covarianzas y extraemos la diagonal

In [12]:
Vrls=np.dot((np.linalg.inv(p)-np.dot(np.dot(np.dot(init,minv),R),np.linalg.inv(p))),np.dot(urls.T,urls))/(n-k-2)
print(Vrls)

[[ 0.83916 -0.0502  -0.00543  0.0006  -0.06521 -0.07671 -0.00264  0.06993
   0.      -0.06993  0.     ]
 [-0.04285  0.00369  0.00029 -0.00009  0.00274  0.00146 -0.00335 -0.00826
   0.0027   0.00444  0.00112]
 [-0.00466  0.00029  0.0002  -0.00017  0.00004 -0.00006 -0.00131 -0.00104
   0.00007  0.00066  0.00031]
 [ 0.00064 -0.00009 -0.00017  0.00063  0.00025  0.00106 -0.00076  0.00047
   0.00078 -0.00068 -0.00058]
 [-0.06326  0.00274  0.00004  0.00025  0.25876  0.0025   0.004    0.01199
  -0.00992 -0.00004 -0.00204]
 [-0.07708  0.00146 -0.00006  0.00106  0.0025   0.09466  0.00972  0.00354
   0.00282 -0.01108  0.00472]
 [-0.00816 -0.00335 -0.00131 -0.00076  0.004    0.00972  0.11071 -0.00473
  -0.01217  0.00827  0.00863]
 [ 0.      -0.00037 -0.00013  0.00012  0.00828  0.00164 -0.00017  0.13986
   0.      -0.06993  0.     ]
 [-0.06993  0.00154  0.00001  0.00049 -0.00664  0.00502 -0.00556  0.06993
   0.06993 -0.06993  0.     ]
 [ 0.      -0.00059  0.00001 -0.00014 -0.00152 -0.00902  0.00012

Calculamos los errores estándar

In [13]:
erls=np.diagonal(Vrls)**(1/2)
print(erls)

[0.91605 0.06074 0.01429 0.02508 0.50868 0.30766 0.33274 0.37398 0.26444
 0.37398 0.26444]


C) Statsmodel sin restricciones

In [15]:
OLS_model = sm.OLS(Y, X)
res = OLS_model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.379
Method:                 Least Squares   F-statistic:                     28.03
Date:                Sun, 23 Aug 2020   Prob (F-statistic):           2.05e-37
Time:                        14:53:14   Log-Likelihood:                -999.32
No. Observations:                 400   AIC:                             2019.
Df Residuals:                     390   BIC:                             2059.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4038      0.711     -1.975      0.0

d) Stats_model constraint: Usaremos por un lado RML y luego GLM ya que ambas tienen la opción de colocar restricciones

In [16]:
resRML = sm.RLM(Y, X, constraints='x7+x8+x9+x10=0').fit()
print(resRML.summary())

                    Robust linear Model Regression Results                    
Dep. Variable:                   wage   No. Observations:                  400
Model:                            RLM   Df Residuals:                      390
Method:                          IRLS   Df Model:                            9
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Sun, 23 Aug 2020                                         
Time:                        14:53:17                                         
No. Iterations:                    30                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5669      0.559     -1.014      0.3

In [17]:
constraints = ['x7+x8+x9+x10=0']
OLS_from_GLM = sm.GLM(Y, X)
resGLM = OLS_from_GLM.fit_constrained(constraints)
print(resGLM.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   wage   No. Observations:                  400
Model:                            GLM   Df Residuals:                      391
Model Family:                Gaussian   Df Model:                            8
Link Function:               identity   Scale:                          8.8821
Method:                          IRLS   Log-Likelihood:                -999.32
Date:                Sun, 23 Aug 2020   Deviance:                       3464.0
Time:                        14:53:43   Pearson chi2:                 3.46e+03
No. Iterations:                     1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7548      0.888     -1.975      0.0

e) Comparamos las respuestas de a y b

In [63]:
x1 = df['educ']
z1 = df['exper']

b1=np.array(b)[np.newaxis]
b1=b1[0,1:3][np.newaxis]
b1reg=1+x1*b1[0,0]+z1*b1[0,1]

print(b1reg)

0       7.430585
1       8.598272
2       7.430585
3       6.928556
4       8.156944
         ...    
395     4.652027
396     4.887401
397     9.333818
398     8.951334
399    10.444519
Length: 400, dtype: float64


In [66]:
x1 = df['educ']
z1 = df['exper']

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111,projection='3d')
ax.plot(x1, z1,Y, 'o',label="data")
ax.plot_surface(x1, z1,b1reg, 'alpha=0.2, cmap='hot')

ax.set_xlabel('Años de educacion')        # Etiqueta del eje X
ax.set_ylabel('Años de experiencia')       # Etiqueta del eje Y
ax.set_zlabel('Salarios');        # Etiqueta del eje Z (Var. Respuesta)


SyntaxError: invalid syntax (<ipython-input-66-fcb679abdfca>, line 7)